## Thuật toán newton lùi cho phương pháp lặp giải đa thức nội suy 

In [12]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_universal_data(filepath):
    """
    Hàm đọc dữ liệu mạnh mẽ, tự động phát hiện định dạng hàng ngang hoặc dọc.
    - Xử lý dấu phẩy (,) và dấu chấm (.).
    - Xử lý ký tự BOM (Byte Order Mark) từ Excel.
    - Bỏ qua các dòng trống hoặc dòng tiêu đề.
    """
    processed_lines = []
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            all_lines = f.readlines()
            for line in all_lines:
                line = line.strip()
                if not line: continue
                line = line.replace(',', '.')
                processed_lines.append(line)
                
    except FileNotFoundError:
        raise IOError(f"LỖI: Không tìm thấy file '{filepath}'.")
    except Exception as e:
        raise IOError(f"LỖI: Không thể đọc file '{filepath}'. Lỗi: {e}")

    if len(processed_lines) == 0: return np.array([])
    elif len(processed_lines) == 1:
        print(f"    (Phát hiện định dạng hàng ngang trong file {filepath})")
        try:
            return np.fromstring(processed_lines[0], sep=' ')
        except Exception as e:
            raise ValueError(f"LỖI: Dữ liệu hàng ngang trong file '{filepath}' không hợp lệ.")
    else:
        print(f"    (Phát hiện định dạng hàng dọc trong file {filepath})")
        data = []
        for i, line in enumerate(processed_lines):
            try:
                value = float(line)
                data.append(value)
            except ValueError:
                print(f"    (Cảnh báo: Bỏ qua dòng {i+1} không hợp lệ: '{line}')")
                pass
        return np.array(data)

# --- Chương trình chính cho Ô nạp dữ liệu ---
try:
    x_coords_raw = load_universal_data('FileX.txt')
    y_coords_raw = load_universal_data('FileY.txt')

    if len(x_coords_raw) == 0 or len(y_coords_raw) == 0:
        raise ValueError("LỖI: Một trong hai file không chứa dữ liệu số hợp lệ.")
    if len(x_coords_raw) != len(y_coords_raw):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
    
    print("Đang sắp xếp dữ liệu theo x giảm dần...")
    sort_indices = np.argsort(x_coords_raw)[::-1]  # Đảo ngược thứ tự (từ lớn đến nhỏ)
    x_coords = x_coords_raw[sort_indices]
    y_coords = y_coords_raw[sort_indices]
    
    diffs = np.diff(x_coords)
    if not np.allclose(diffs, diffs[0], rtol=1e-08, atol=1e-10):
        raise ValueError(f"LỖI: Các mốc nội suy x không cách đều nhau. Phương pháp lặp không thể áp dụng.")
    
    h_step = abs(diffs[0])  # Lưu lại bước nhảy h (luôn dương)

    print("Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều THÀNH CÔNG!")
    print(f"Tổng số {len(x_coords)} điểm đã được nạp. Bước nhảy h = {h_step:g}")
    print("-" * 50)
    print("Các mốc x (đã sắp xếp giảm dần):", x_coords.tolist())
    print("Các giá trị y (tương ứng):", y_coords.tolist())
    
except (IOError, ValueError) as e:
    print(str(e))

    (Phát hiện định dạng hàng dọc trong file FileX.txt)
    (Phát hiện định dạng hàng dọc trong file FileY.txt)
Đang sắp xếp dữ liệu theo x giảm dần...
Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều THÀNH CÔNG!
Tổng số 7 điểm đã được nạp. Bước nhảy h = 0.118
--------------------------------------------------
Các mốc x (đã sắp xếp giảm dần): [7.726, 7.608, 7.49, 7.372, 7.254, 7.136, 7.018]
Các giá trị y (tương ứng): [3.2679, 2.8701, 2.4553, 2.0293, 1.5979, 1.1671, 0.7427]


In [13]:
def create_full_difference_table(y_values):
    """Tạo bảng sai phân (sử dụng ∇ thay vì Δ): ∇^j y_i = ∇^{j-1} y_i - ∇^{j-1} y_{i+1}."""
    num_points = len(y_values)
    table = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    
    # Cột đầu tiên là y
    for i in range(num_points):
        table[i][0] = y_values[i]
        
    # Tính các cột sai phân theo công thức y_k - y_{k+1}
    for j in range(1, num_points):  # Cột (bậc)
        for i in range(num_points - j):  # Hàng
            # Công thức sai phân "lùi" (hoặc đảo dấu): ∇^j y_i = ∇^{j-1} y_i - ∇^{j-1} y_{i+1}
            table[i][j] = table[i][j-1] - table[i+1][j-1]
            
    return np.array(table)

# --- Chương trình chính cho Ô 2 ---
try:
    # Tính bảng sai phân đầy đủ
    diff_table_for_calc = create_full_difference_table(y_coords)
    
    print("--- Bảng sai phân đầy đủ (dạng tam giác trên) ---")
    display(pd.DataFrame(diff_table_for_calc).style.format("{:g}", na_rep=""))
    print("Bảng sai phân đã được tính và lưu vào 'diff_table_for_calc'.")

except (NameError) as e:
    print(f"LỖI: {e}. Vui lòng chạy lại Ô 1 trước.")

--- Bảng sai phân đầy đủ (dạng tam giác trên) ---


,0,1,2,3,4,5,6
0,3.2679,0.3978,-0.017,-0.0058,-8.88178e-16,-0.0002,-0.0006
1,2.8701,0.4148,-0.0112,-0.0058,0.0002,0.0004,0
2,2.4553,0.426,-0.0054,-0.006,-0.0002,0,0
3,2.0293,0.4314,0.0006,-0.0058,0,0,0
4,1.5979,0.4308,0.0064,0,0,0,0
5,1.1671,0.4244,0,0,0,0,0
6,0.7427,0,0,0,0,0,0


Bảng sai phân đã được tính và lưu vào 'diff_table_for_calc'.


In [14]:
def psi_function(t, y_bar, coeffs_row_k):
    """
    Tính giá trị của hàm lặp psi(t) dựa trên công thức Newton lùi.
    coeffs_row_k là mảng [y_k, ∇y_k, ∇^2y_k, ...]
    Dữ liệu giảm dần (y_0 > y_1 > ... > y_n)
    """
    y_k = coeffs_row_k[0]
    nabla_y_k = coeffs_row_k[1]
    
    if np.isclose(nabla_y_k, 0):
        return np.nan 
    
    # Tính phần trong ngoặc [...]
    # Công thức Newton lùi: t(t-1), t(t-1)(t-2), ...
    bracket_sum = 0.0
    
    for i in range(2, len(coeffs_row_k)):
        # Tính tích t(t-1)...(t-i+1)
        t_product = 1.0
        for j in range(i):
            t_product *= (t - j)
        
        # Thêm số hạng (∇^i y_k / i!) * t_product
        if i < len(coeffs_row_k):
            bracket_sum += (coeffs_row_k[i] / math.factorial(i)) * t_product
        
    # Áp dụng công thức lặp
    t_new = (y_bar - y_k) / nabla_y_k - (1 / nabla_y_k) * bracket_sum
    
    return t_new

print("Hàm lặp psi(t) (Newton lùi - y giảm dần) đã được cập nhật.")

Hàm lặp psi(t) (Newton lùi - y giảm dần) đã được cập nhật.


In [15]:
def print_psi_formula(y_bar, coeffs_row_k_plus_1):
    """
    In ra công thức tượng trưng của hàm lặp psi(t).
    """
    print("\n" + "="*50)
    print("--- CÔNG THỨC LẶP ψ(t) TƯƠNG ỨNG ---")
    
    y_k_plus_1 = coeffs_row_k_plus_1[0]
    nabla_y_k_plus_1 = coeffs_row_k_plus_1[1]
    
    if np.isclose(nabla_y_k_plus_1, 0):
        print("LỖI: ∇y_{k+1} bằng 0, không thể xây dựng công thức lặp.")
        return

    term_t0 = (y_bar - y_k_plus_1) / nabla_y_k_plus_1
    term_multiplier = -1.0 / nabla_y_k_plus_1
    
    formula_str = f"ψ(t) = {term_t0:g}"
    
    bracket_str = ""
    for i in range(2, len(coeffs_row_k_plus_1)):
        diff_val = coeffs_row_k_plus_1[i]
        if np.isclose(diff_val, 0):
            continue
        term_coeff = diff_val / math.factorial(i)
        sign = " - " if term_coeff < 0 else " + "
        if bracket_str == "":
            sign = "-" if term_coeff < 0 else ""
            
        # Xây dựng tích t(t+1)...(t+i-1)
        t_product_terms = []
        for j in range(i):
            if j == 0:
                t_product_terms.append("t")
            else:
                t_product_terms.append(f"(t+{j})")
        t_product_str = "*".join(t_product_terms)
        
        bracket_str += f"{sign} {abs(term_coeff):g} * {t_product_str} "
        
    if bracket_str:
        sign_multiplier = " - " if term_multiplier < 0 else " + "
        formula_str += f"{sign_multiplier} {abs(term_multiplier):g} * [ {bracket_str}]"
        
    print(formula_str)
    print("="*50)

print("Hàm in công thức lặp 'print_psi_formula' đã được định nghĩa.")

Hàm in công thức lặp 'print_psi_formula' đã được định nghĩa.


In [16]:
try:
    max_iterations = 1000000
    tolerance = 1e-5
    
    y_prime_input = float(input(f"Nhập giá trị y (y_bar) bạn muốn tìm x: "))
    
    print(f"\nThông tin dữ liệu hiện tại:")
    print(f"  Y giảm dần: [{y_coords[0]:g}, {y_coords[-1]:g}]")
    print(f"  Các giá trị y: {y_coords.tolist()}")
    
    # --- 2. Tìm khoảng cách ly (y giảm dần) ---
    def find_isolation_interval(y_arr, y_val, tol=1e-6):
        """
        Tìm chỉ số k sao cho:
        - y_arr[k] <= y_val <= y_arr[k+1] nếu y tăng dần
        - y_arr[k] >= y_val >= y_arr[k+1] nếu y giảm dần
        In ánh xạ (i, x_i, y_i) để kiểm tra.
        """
        n = len(y_arr)
        if n < 2:
            return -1

        # In ánh xạ để kiểm tra
        try:
            print("Ánh xạ chỉ mục (i, x_i, y_i):")
            for i, (xx, yy) in enumerate(zip(x_coords, y_coords)):
                print(f"  i={i}: x={xx:g}, y={yy:g}")
        except NameError:
            pass

        dy = np.diff(y_arr)
        is_increasing = np.all(dy >= -tol)
        is_decreasing = np.all(dy <= tol)

        if not (is_increasing or is_decreasing):
            print("LỖI: Dữ liệu y không đơn điệu. Không thể tìm khoảng cách ly tự động.")
            return -1

        if is_increasing:
            # tăng dần
            if y_val < y_arr[0] - tol or y_val > y_arr[-1] + tol:
                print("❌ y ngoài range (tăng).")
                return -1
            for k in range(n - 1):
                if (y_arr[k] - tol) <= y_val <= (y_arr[k + 1] + tol):
                    print(f"✓ Tìm thấy (tăng): k={k}")
                    return k
        else:
            # giảm dần
            if y_val < y_arr[-1] - tol or y_val > y_arr[0] + tol:
                print("❌ y ngoài range (giảm).")
                return -1
            for k in range(n - 1):
                if (y_arr[k + 1] - tol) <= y_val <= (y_arr[k] + tol):
                    print(f"✓ Tìm thấy (giảm): k={k}")
                    return k

        return -1
    
    k_index = find_isolation_interval(y_coords, y_prime_input, tol=1e-6)
    
    if k_index == -1:
        raise ValueError(f"LỖI: y={y_prime_input} ngoài khoảng [{y_coords[-1]:g}, {y_coords[0]:g}]")
    
    # --- 3. Lấy hệ số từ hàng k ---
    start_node_index = k_index
    if start_node_index >= len(diff_table_for_calc):
        start_node_index = len(diff_table_for_calc) - 1
    
    x_k = x_coords[start_node_index]
    y_k = y_coords[start_node_index]
    backward_coeffs_k = diff_table_for_calc[start_node_index, :]
    
    print(f"\n✓ y={y_prime_input:g} trong khoảng k={k_index}")
    print(f"==> Mốc: x_k+1 = {x_k:g}, y_k+1 = {y_k:g}")
    print(f"Hệ số (hàng {start_node_index}): {backward_coeffs_k}")
    
    print_psi_formula(y_prime_input, backward_coeffs_k)

    # --- 4. Vòng lặp ---
    nabla_y_k = backward_coeffs_k[1]
    if np.isclose(nabla_y_k, 0):
        raise ValueError("LỖI: ∇y_k = 0")
        
    t_old = (y_prime_input - y_k) / nabla_y_k
    print(f"\n--- Bắt đầu lặp với t_0 = {t_old:g} ---")
    
    iteration_log = [[0, t_old, np.nan]]
    converged = False
    x_result = np.nan
    t_final = t_old

    for j in range(max_iterations):
        t_new = psi_function(t_old, y_prime_input, backward_coeffs_k)
        
        if np.isnan(t_new):
            print(f"❌ Lỗi tính toán psi(t)")
            break
            
        error = abs(t_new - t_old)
        iteration_log.append([j + 1, t_new, error])
        
        if error < tolerance:
            print(f"✓ Hội tụ sau {j+1} lần lặp")
            converged = True
            t_final = t_new
            x_result = x_k + t_final * h_step
            break
            
        t_old = t_new
        
    # --- 5. Bảng lặp ---
    print("\n" + "="*50)
    print("BẢNG LỊCH SỬ LẶP")
    df_iterations = pd.DataFrame(iteration_log, columns=['k', 't_k', '|Δt|'])
    display(df_iterations.style.format("{:g}", subset=['t_k', '|Δt|'], na_rep="---"))
    
    # --- 6. Kết quả ---
    if converged:
        print("\n" + "="*50)
        print("KẾT QUẢ NỘI SUY NGƯỢC (NEWTON LÙI)")
        print(f"t cuối: {t_final:g}")
        print(f"x = {x_k:g} + {t_final:g} * {h_step:g} = {x_result:g}")
        print(f">>> Kết quả: x ≈ {x_result:g} <<<")
        print("="*50)
    else:
        print(f"\n❌ Không hội tụ")

except (ValueError, NameError, IOError) as e:
    print(f"❌ {e}")


Thông tin dữ liệu hiện tại:
  Y giảm dần: [3.2679, 0.7427]
  Các giá trị y: [3.2679, 2.8701, 2.4553, 2.0293, 1.5979, 1.1671, 0.7427]
Ánh xạ chỉ mục (i, x_i, y_i):
  i=0: x=7.726, y=3.2679
  i=1: x=7.608, y=2.8701
  i=2: x=7.49, y=2.4553
  i=3: x=7.372, y=2.0293
  i=4: x=7.254, y=1.5979
  i=5: x=7.136, y=1.1671
  i=6: x=7.018, y=0.7427
✓ Tìm thấy (giảm): k=0

✓ y=3 trong khoảng k=0
==> Mốc: x_k+1 = 7.726, y_k+1 = 3.2679
Hệ số (hàng 0): [ 3.2679000e+00  3.9780000e-01 -1.7000000e-02 -5.8000000e-03
 -8.8817842e-16 -2.0000000e-04 -6.0000000e-04]

--- CÔNG THỨC LẶP ψ(t) TƯƠNG ỨNG ---
ψ(t) = -0.673454 -  2.51383 * [ - 0.0085 * t*(t+1)  -  0.000966667 * t*(t+1)*(t+2)  -  1.66667e-06 * t*(t+1)*(t+2)*(t+3)*(t+4)  -  8.33333e-07 * t*(t+1)*(t+2)*(t+3)*(t+4)*(t+5) ]

--- Bắt đầu lặp với t_0 = -0.673454 ---
✓ Hội tụ sau 4 lần lặp

BẢNG LỊCH SỬ LẶP


,k,t_k,|Δt|
0,0,-0.673454,---
1,1,-0.656296,0.0171575
2,2,-0.656867,0.000570635
3,3,-0.656848,1.88772e-05
4,4,-0.656849,6.24591e-07



KẾT QUẢ NỘI SUY NGƯỢC (NEWTON LÙI)
t cuối: -0.656849
x = 7.726 + -0.656849 * 0.118 = 7.64849
>>> Kết quả: x ≈ 7.64849 <<<
